## Let's try to build a first colection, json format, of the iVerifyZambia csv. We will use whoosh to create an index and we will try to perform some partial tests searching queries.

In [1]:
import pandas as pd
import json

In [2]:
data=pd.read_csv('iverifyzm.csv')

In [3]:
data

,Unnamed: 0,claims,evidences,label,date
0,0,"On March 2, 2023, Koswe Facebook page publish...",The iVerify Zambia has determined as false the...,False,"Mar, 09 2023"
1,1,"On 22 February 2023, a Facebook page called Za...",iVerify has determined as true reports that Cy...,True,"Feb, 23 2023"
2,2,"On February 21, 2023, a Facebook page called K...",iVerify Zambia has determined as misleading th...,Misleading,"Feb, 23 2023"
3,3,"The Patriotic Front (PF) Facebook page, on Jan...",iVerify Zambia has determined as false the cla...,False,"Feb, 22 2023"
4,4,"On Wednesday, February 17, 2023, Grindstone Te...",iVerify Zambia has determined as misleading th...,Misleading,"Feb, 18 2023"
...,...,...,...,...,...
206,206,"On 13th October, 2021, The Candidates uploaded...",The Fact Checking process has determined as tr...,True,"Oct, 20 2021"
207,207,"On 28th July 2021, News Diggers Newspaper publ...",The fact checking process has determined as mi...,Misleading,"Aug, 04 2021"
208,208,The Patriotic Front (PF) Facebook Page publish...,The Fact Checking Process has determined as fa...,False,"Nov, 12 2021"
209,209,"On 12th July, 2021, the Smart Eagles Facebook ...",The fact checking process has determined the c...,Partly False,"Jul, 20 2021"


In [4]:
def create_json(name,pjson):
    with open(f'{name}','w') as jf:
        json.dump(pjson,jf)

### We are going to use a pretrained model called Voicelab NLP model in order to get the keywords of the evidences for each row of the dataset, in order to maximize the efficiency of search later on in our collection.

In [ ]:
#!pip install transformers

In [8]:
#!pip install torch
#!pip install sentencepiece

In [5]:
ev=data['evidences']

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("Voicelab/vlt5-base-keywords")
tokenizer = T5Tokenizer.from_pretrained("Voicelab/vlt5-base-keywords")
keywords=[]

task_prefix = "Keywords: "


for sample in ev:
    input_sequences = [task_prefix + sample]
    input_ids = tokenizer(
        input_sequences, return_tensors="pt", truncation=True
    ).input_ids
    output = model.generate(input_ids, no_repeat_ngram_size=2, num_beams=4)
    predicted = tokenizer.decode(output[0], skip_special_tokens=True)
    
    #print(sample, "\n --->", predicted)
    keywords.append(predicted)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
C:\Users\Pepe\anaconda3\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [8]:
keywords

['Koswe Facebook page, iVerify Zambia',
 'Decaster Management and Mitigation Unit (DMMU), Cyclone Freddy',
 'Kalbalika Facebook page, iVerify Zambia, sexuality e',
 'Zambia, iVerify, pit copper mine',
 'ZESCO Spokesperson, iVerify Zambia, social media',
 'Balmoral Police post in Makeni, iVerify Zambia, snake',
 'iVerify Zambia, party Presidency, watchdog Facebook',
 'Alick Banda, Lusaka Archdiocese, iVerify Zambia,',
 'DEC, Crest lodge, exhausting the notice of seizure',
 'Facebook Page, iVerify Zambia, fact checking',
 'Chitalu Chilufy, Zambian Bulletin Facebook page',
 'Chibamba Kanyama, Zambia, United States of America (USA), iVeri',
 'Zambia Revenue Authority, iVerify, social media',
 'appointment of Kangwa Chileshe as Permanent Secretary,',
 'Lusaka City Council, iVerify Zambia, contact numbers',
 'Kariba Dam, dry rock bed downstream, low wa',
 'Kariba Dam, dry rock bed downstream, low wa',
 'Kazungula Bridge, iVerify Zambia',
 'National Pension Scheme Authority (NAPSA), iVerify Z

In [9]:
pre_json={}
i=1
c=0
for index,row in data.iterrows():
    
    date=row['date']
    pre_json['claim']=row['claims']
    pre_json['evidences']=row['evidences']
    pre_json['label']=row['label']
    pre_json['date']=row['date']
    pre_json['keywords']=keywords[c].split(',')
    fullname=f'collection/{i}'
    create_json(fullname,pre_json)
    
    c+=1  
    i+=1

### By now we have a directory called collection in which we have for each row of the dataframe on json file including the claim, the evidences, the label, the date and the keywords we sustracted thanks to the Voicelab NLP model